In [ ]:
from canvasapi import Canvas # pip install canvasapi

from autocanvas.config import INPUT_DIR

from autocanvas.core.conversions import (
    series_from_api_object, 
    df_from_api_list)

from autocanvas.core.course_info import (
    get_PHY_course, 
    get_assignment_group_from_name, 
    get_teaching_personel,
    get_sections,
    get_students_from_sections,)

from autocanvas.core.assignments import get_assignment_collection

from os.path import join
import re
import pandas as pd     # manipulate tabulated data
pd.set_option('display.max_columns', 500)
from datetime import datetime 


API_URL = "https://ufl.instructure.com/"
try:
    from autocanvas.config import get_API_key
    API_KEY = get_API_key()
except FileNotFoundError as e:
    print(e)
    API_KEY = input("Asking for API token")

canvas = Canvas(API_URL, API_KEY)

In [ ]:
course = get_PHY_course(canvas, 
                        course_code="PHY2053", 
                        semester="Summer 2021")
print(course.name)

In [ ]:
df_TAs, df_teachers = get_teaching_personel(course, add_first_name=True, 
                          groups=["ta", "teacher"])

file_name = "section_ta_phy2053_summer2021.csv"
file_path = join(INPUT_DIR, file_name)

df_students, df_sections = get_students_from_sections(
                                course, 
                                section_ta_csv=file_path)

In [ ]:
df_students

# Get student Emails

In [ ]:
import aiohttp
import asyncio
async def gather_emails():
    async def get_user_emails(user_id):
        async with aiohttp.ClientSession() as session:
            async with session.get(API_URL+"api/v1/"+"users/{}/profile/".format(user_id),
                                      headers={'Authorization': 'Bearer {}'.format(API_KEY)}) as resp:
                if resp.status==200:
                    all_user_emails[user_id] = await resp.json()
                else:
                    await resp.text()
                    print(resp.status)
    # need to deal with high watermark:
    # https://community.canvaslms.com/t5/Developers-Group/API-Rate-Limiting/ba-p/255845
    # suggestion of groups of 30 every 0.250 seconds:
    # https://community.canvaslms.com/t5/Developers-Group/API-Rate-Limiting/m-p/211140
    rate_limit = 30
    all_user_emails = {}
    len_students = len(df_students)
    bunchstart = 0
    while bunchstart < len_students:
        print(bunchstart)
        coros = [get_user_emails(user_id) for user_id in df_students.id[bunchstart:bunchstart+rate_limit]]
        await asyncio.gather(*coros)
        await asyncio.sleep(0.250)
        bunchstart += rate_limit 
        
    return all_user_emails

all_user_emails = await gather_emails()

In [ ]:
df_emails = (pd.DataFrame(all_user_emails)
             .T
             [["name", "login_id", "id"]]
             .set_index("name")
             .rename(columns={"login_id":"email"})
            )
df_emails

In [ ]:
df_students = pd.merge(df_emails, df_students, 
                         how="inner", 
                         left_on="id",
                         right_on="id",
                        validate="1:1")
df_students

# Select Week

In [ ]:
week_number = 3
class_number = 18215
min_attendance = {
    "T":20, 
    "R":50
}

In [ ]:
df_all_students = (df_students[df_students["class_number"]==class_number]
                     [["class_number", "name", "sortable_name","email"]]
                  ).set_index("name").copy()
df_all_students

# Using Student Emails

In [ ]:
df_all_students = (df_students[df_students["class_number"]==class_number]
                     [["class_number", "name", "sortable_name","email"]]
                  ).set_index("name").copy()
df_all_students["is_member"] = 1
attended_external = {}
for day in min_attendance.keys():
    fname = "week_{}_section_{}_{}.csv".format(week_number, 
                                                  class_number, 
                                                  day)
    excel_path = join(INPUT_DIR, fname)
    df_participants = (pd.read_csv(excel_path, 
                            skiprows=3,
                            skipfooter=0,
                            engine="python")
                       .set_index("User Email")
               )
    len_participants = len(df_participants)
#     print(df_participants.columns)
    df_session_info = (pd.read_csv(excel_path, 
                                   engine="python",
                                   skipfooter=len_participants+1)
        )
    points_col = "points_{}".format(day)
    df_participants[points_col] = (df_participants.filter(regex=".*Duration \(Minutes\)", axis=1)
                                    > min_attendance[day]
                                )*1
    # return indices of self not in other
    attended_external[day] = (df_participants[points_col].index
                      .difference(df_all_students.set_index("email").index))
    for item in  attended_external[day]:
        print(item, df_participants[points_col][item])
    df_all_students = pd.merge(df_all_students, df_participants[points_col],
             how="left", left_on="email", right_index=True)
#     print(df_all_students["email"])
#     print(df_all_students["email"].compare(df_participants.index))
points_columns = ["points_{}".format(day) 
                  for day in min_attendance.keys()]
df_all_students["total_points"] = df_all_students[points_columns].sum(axis=1)

# we want case-insensitive sorting
total_from_emails = df_all_students.iloc[df_all_students.sortable_name
                     .str.lower().argsort()]["total_points"]

print(attended_external)
total_from_emails


# (OLD) Using Student Names 

This proved to be error-prone, because some students do not have the same name in Zoom and in Canvas

In [ ]:
df_all_students = (df_students[df_students["class_number"]==class_number]
                     [["class_number", "name", "sortable_name","email"]]
                  ).set_index("name").copy()
for day in min_attendance.keys():
    fname = "week_{}_section_{}_{}.csv".format(week_number, 
                                                  class_number, 
                                                  day)
    excel_path = join(INPUT_DIR, fname)
    df_participants = (pd.read_csv(excel_path, 
                            skiprows=3,
                            skipfooter=0,
                            engine="python")
                       .set_index("Name (Original Name)")
               )
    len_participants = len(df_participants)
    df_session_info = (pd.read_csv(excel_path, 
                                   engine="python",
                                   skipfooter=len_participants+1)
        )
    points_col = "points_{}".format(day)
    
    df_participants[points_col] = (df_participants.filter(regex=".*Duration \(Minutes\)", axis=1)
                                    > min_attendance[day]
                                )*1
    df_all_students = pd.merge(df_all_students, df_participants[points_col],
             how="left", left_index=True, right_index=True)
points_columns = ["points_{}".format(day) 
                  for day in min_attendance.keys()]
df_all_students["total_points"] = df_all_students[points_columns].sum(axis=1)

# we want case insensitive sorting
total_from_names = df_all_students.iloc[df_all_students.sortable_name
                                         .str.lower().argsort()]["total_points"]
total_from_names

In [ ]:
total_from_emails.compare(total_from_names)